The `ipython_blocking` package allows the capture of cell execution, effectively 'blocking' further action until you release.  The original intended use-case is to block until a widget value has changed or other sort of validation of widget input has been done.

In [1]:
import ipython_blocking
import ipywidgets as widgets

The simplest example is blocking further cell execution until a widget value has changed.  Run the next three cells which create a widget, start the 'blocking' context, then print the widget value *only after the widget value has changed to something other than an empty string*.

In [2]:
dd = widgets.Dropdown(options=['', 'foo', 'bar', 'baz'])
dd

Dropdown(options=('', 'foo', 'bar', 'baz'), value='')

In [18]:
ctx = ipython_blocking.CaptureExecution()
with ctx:
    while True:
        if dd.value:
            break
        ctx.step()

In [4]:
print(dd.value)

foo


The explicit context creation above is also wrapped into an IPython line magic called `%block`.  That magic expects one positional argument, which should either be the variable name of a single widget or an arbitrary function.

If the positional argument is a single widget, the `CaptureExecution` context will stop when that widget changes value.  If the positional argument is a function, then it will stop when the called function returns `True`.  Be careful that the function doesn't do anything very complicated since it will be called very often.

The magic command also accepts a single optional `-t` or `--timeout` argument, which is a time in seconds until the context ends on its own.  The default is to not have a timeout.

In [5]:
ipython_blocking.load_ipython_extensions()

The next three lines are nearly identical to the first but will capture cell execution until the `dd2` widget value has changed.

In [6]:
dd2 = widgets.Dropdown(options=['one', 'two', 'three'])
dd2

Dropdown(options=('one', 'two', 'three'), value='one')

In [8]:
%block dd2

In [9]:
print(dd2.value)

three


Next is an example of using the `%block` magic with a more arbitrary validation function.  This will not exit the cell execution capture context until the `dd3` widget is set to value `baz`.

In [10]:
dd3 = widgets.Dropdown(options=['', 'foo', 'bar', 'baz'])
dd3

Dropdown(options=('', 'foo', 'bar', 'baz'), value='')

In [12]:
def baz_validation():
    return dd3.value == 'baz'

%block baz_validation

In [13]:
print(dd3.value)

baz


Finally, an example of a more complex validation function.  This last example will block cell execution until the "input text" box has at least 14 characters and the "Submit" button has been pressed.

In [14]:
text = widgets.Text(description='input text:')
submit = widgets.Button(description="Submit", button_style='info')
submit.submitted = False # use a widget name-space attribute so it can be referenced in on_click
box = widgets.VBox(children=[text, submit])

def check_text_length(ev):
    if len(text.value) < 14:
        submit.button_style = 'info'
    else:
        submit.button_style = 'success'

def on_submit(ev):
    if len(text.value) >= 14:
        submit.submitted = True
        submit.disabled = True # make sure they can't submit twice

text.observe(check_text_length)        
submit.on_click(on_submit)
break_func = lambda: submit.submitted # i.e. function to return whether submitted is True or False

box

VBox(children=(Text(value='', description='input text:'), Button(button_style='info', description='Submit', style=ButtonStyle())))

In [16]:
%block break_func

In [17]:
print(text.value)

one two three x y z
